<a href="https://colab.research.google.com/github/wdon021/From_Colab/blob/master/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
PATH = "/content/gdrive/My Drive/Fastai/"
import pandas as pd
h1b_raw = pd.read_csv(f'{PATH}H-1B_Disclosure_RAW_Data_FY18.csv', delimiter=',', engine='python')

In [0]:
import re

In [0]:
def removeAllPunctuations(g):
  g = g.str.replace(".", " ")
  g= g.str.replace(","," ")
  g= g.str.replace("'","")
  g= g.str.replace("-"," ")
  g= g.str.replace("/"," ")
  g= g.str.replace(":"," ")
  g= g.str.replace(";"," ")
  g= g.str.replace('"',"")
  g= g.str.replace("*","")
  g= g.str.replace("?"," ")
  g= g.str.replace("&"," and ")
  g= g.str.replace("+"," ")
  g= g.str.replace("["," ")
  g= g.str.replace("]"," ")
  g= g.str.replace("("," ")
  g= g.str.replace(")"," ")
  g= g.str.replace("<"," ")
  g= g.str.replace(">"," ")
  g= g.str.replace("="," ")
  g= g.str.replace(","," ")
  g= g.str.replace("{"," ")
  g= g.str.replace("}"," ")  
  # g= re.sub( '\s+', ' ', g ).strip()
  return g

In [0]:
h1b_raw['JOB_TITLE'] = removeAllPunctuations(h1b_raw['JOB_TITLE'])

In [0]:
job = list(h1b_raw['JOB_TITLE'].str.lower().astype('category').cat.categories)

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [0]:
from nltk import word_tokenize

In [0]:
import nltk

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from gensim.models import word2vec, Phrases

In [0]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(job)]

In [0]:
max_epochs = 100
vec_size = 20
alpha = 0.025

In [17]:
model = Doc2Vec(size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm=1) # 1 preseres the word order, 0 doesn't preserve any word order

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [1]:
model.build_vocab(tagged_data)

NameError: ignored

In [2]:
for epoch in range(max_epochs):
  print('iteration {0}'.format(epoch))
  model.train(tagged_data,
              total_examples=model.corpus_count,
              epochs=model.iter)
  # decrease the learning rate
  model.alpha -= 0.0002
  
model.save('d2v.model')
print("model Saved")

NameError: ignored

In [0]:
model.save(f'{PATH}d2v.model')

In [18]:
model = Doc2Vec.load(f'{PATH}d2v.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
job_vec = list()

In [0]:
for jobvec in range(0, len(job)):
  job_vec.append(model.docvecs[jobvec])

In [47]:
job_vec

[array([-0.3761064 ,  0.48634037,  0.3828929 , -0.7684472 , -0.20532824,
        -0.20266764,  0.39691973,  0.48283175,  0.8440014 , -1.1450241 ,
        -0.23301597,  0.6559546 ,  0.61935955, -0.22326425,  0.57737195,
        -0.7944947 , -0.17471246,  1.0284306 ,  0.51423836,  0.28983638],
       dtype=float32),
 array([-0.5901237 , -0.9342211 ,  0.6940091 ,  0.12525664, -0.5113273 ,
        -0.06945222,  0.20241532,  1.0578412 ,  0.9255435 , -0.14961387,
        -0.531674  , -0.04779436,  0.6126967 ,  1.0476264 ,  0.694198  ,
        -1.2181737 ,  0.63418454, -0.642101  , -0.08569869,  1.2350522 ],
       dtype=float32),
 array([ 0.6661586 , -1.1289634 , -0.08642422, -1.0076795 ,  0.06378692,
         1.0880959 ,  0.528954  ,  0.10642952,  0.05796468, -1.4576914 ,
         0.07417589,  2.1026416 ,  1.5370783 ,  0.5555942 , -0.24364181,
        -0.22024025,  1.3306592 ,  0.8841772 , -0.47921005,  0.5872402 ],
       dtype=float32),
 array([ 0.42972133,  1.086924  ,  1.3753645 ,  0.27

In [31]:
model.docvecs[1]

array([-0.5901237 , -0.9342211 ,  0.6940091 ,  0.12525664, -0.5113273 ,
       -0.06945222,  0.20241532,  1.0578412 ,  0.9255435 , -0.14961387,
       -0.531674  , -0.04779436,  0.6126967 ,  1.0476264 ,  0.694198  ,
       -1.2181737 ,  0.63418454, -0.642101  , -0.08569869,  1.2350522 ],
      dtype=float32)

In [0]:
docvec = model.docvecs[1]

In [0]:
print(docvec)

In [0]:
similar_doc = model.docvecs.most_similar('0')

In [0]:
model.vocabulary.

In [0]:
job['66796']

In [0]:
print(similar_doc)

In [0]:
trigram_sentences_project = []
bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')

In [0]:
for sent in sentence_stream:
  bigrams_ = bigram[sent]
  trigrams_ = trigram[bigram[sent]]
  trigram_sentences_project.append(trigrams_)

In [0]:
num_features = 10 # Word vector demensionality
min_word_count = 1 # minumum word count
num_workers = 4 # number of threads to run in parallel
context = 5 # context window size
downsampling = 1e-3 #downsample setting for frequent 

In [0]:
model = word2vec.Word2Vec(min_count=10, window=2, size=100, sample=6e-5, alpha=0.05, min_alpha=0.0007, negative=20)

In [0]:
model.build_vocab(trigram_sentences_project, progress_per=1000)

In [0]:
model.train(trigram_sentences_project, total_examples=model.corpus_count, epochs=30, report_delay=1)

In [0]:
model.wv.most_similar('scientist')

In [0]:
vocab = list(model.wv.vocab.keys())

In [0]:
print(vocab[:10])

In [0]:
job[0]

In [0]:
trigram_sentences_project

In [0]:
model.wv['&nbsp;application integration']

In [0]:
job[0]

In [0]:
for :
  print(i)